In [62]:
import pandas as pd
import numpy as np
import chess
from tqdm.notebook import tqdm

NAME = "Carlsen, Magnus"
name_file = 

SyntaxError: invalid syntax (Temp/ipykernel_9564/1142265103.py, line 7)

In [63]:
games = pd.read_csv("archive/chess/games_chesstempo.csv")
games.head()

,id_game,moves,date,white_player,id_white_player,black_player,id_black_player,white_elo,black_elo,avg_elo,event,site,eco,opening,id_opening,round,result,n_moves,id_download
0,https://old.chesstempo.com/gamedb/game/4820223,1.e4 e5 2.Nf3 Nc6 3.Bc4 Bc5 4.c3 Nf6 5.d4 exd4...,2021-05-13,"Abasov, Nijat Azad Oglu",https://old.chesstempo.com/gamedb/player/248,"Safarli, Eltaj",https://old.chesstempo.com/gamedb/player/183497,2665.0,2598.0,2631.0,Azerbaijan Championship,asf.org.az,C53,"Italian Game, Classical Variation, Greco Gambit",https://old.chesstempo.com/gamedb/opening/869,10,1-0,36.0,https://old.chesstempo.com/requests/download_g...
1,https://old.chesstempo.com/gamedb/game/4819358,1.e4 c5 2.Nf3 d6 3.d4 cxd4 4.Nxd4 Nf6 5.Nc3 a6...,2021-05-11,"Abasov, Nijat Azad Oglu",https://old.chesstempo.com/gamedb/player/248,"Muradli, Mahammad",https://old.chesstempo.com/gamedb/player/316504,2665.0,2449.0,2557.0,Azerbaijan Championship,asf.org.az,B94,"Sicilian Defense, Najdorf Variation",https://old.chesstempo.com/gamedb/opening/2149,8,1-0,21.0,https://old.chesstempo.com/requests/download_g...
2,https://old.chesstempo.com/gamedb/game/4818704,1.d4 Nf6 2.c4 g6 3.Nc3 Bg7 4.e4 d6 5.Be2 O-O 6...,2021-05-09,"Abasov, Nijat Azad Oglu",https://old.chesstempo.com/gamedb/player/248,"Iskandarov, Misratdin Mehdi oglu",https://old.chesstempo.com/gamedb/player/92096,2665.0,2535.0,2600.0,Azerbaijan Championship,asf.org.az,E73,"King's Indian Defense, Normal Variation, Stand...",https://old.chesstempo.com/gamedb/opening/1195,6,0-1,37.0,https://old.chesstempo.com/requests/download_g...
3,https://old.chesstempo.com/gamedb/game/4818558,1.d4 Nf6 2.c4 e6 3.g3 d5 4.Bg2 Be7 5.Nf3 O-O 6...,2021-05-08,"Abbasov, Farid",https://old.chesstempo.com/gamedb/player/314,"Abasov, Nijat Azad Oglu",https://old.chesstempo.com/gamedb/player/248,2528.0,2665.0,2596.0,Azerbaijan Championship,asf.org.az,E06,"Catalan Opening, Closed Variation",https://old.chesstempo.com/gamedb/opening/317,5,½-½,32.0,https://old.chesstempo.com/requests/download_g...
4,https://old.chesstempo.com/gamedb/game/4820371,1.e4 e5 2.Nf3 Nf6 3.Nxe5 d6 4.Nf3 Nxe4 5.c4 Nc...,2021-05-07,"Abasov, Nijat Azad Oglu",https://old.chesstempo.com/gamedb/player/248,"Garayev, Kanan",https://old.chesstempo.com/gamedb/player/316060,2665.0,2331.0,2498.0,Azerbaijan Championship,asf.org.az,C42,"Russian Game, Kaufmann Attack",https://old.chesstempo.com/gamedb/opening/1871,4,1-0,26.0,https://old.chesstempo.com/requests/download_g...


In [64]:
games = games[games["white_player"] == NAME]
games.shape

(1474, 19)

In [65]:
def extract_moves(games):
    moves = []
    for col_moves in games["moves"]:
        col_moves = col_moves.split(' ')
        moves.append([i if not i[0:1].isdigit() else i.split('.')[1] for i in col_moves][:-1])
    for i, move in tqdm(enumerate(moves)):
        for j, m in enumerate(move):
            if "e.p" in m:
                moves[i].pop(j)
    return moves

In [66]:
def from_str_to_int_win(win, color):
    if '1-0' == win:
        return 1
    elif '0-1' == win:
        return 0
    return 0.5


In [67]:
def extract_game_fen(moves, win, positions, times_played, number_of_win, color):
    board = chess.Board()
    win_int = from_str_to_int_win(win, color)
    for i, move in enumerate(moves):
        board.push_san(move)
        if((i+color) % 2):
            fen = board.epd()
            if fen in positions:
                index = positions.index(fen)
                times_played[index]+=1
                number_of_win[index]+=win_int
            else:
                positions.append(fen)
                number_of_win.append(win_int)
                times_played.append(1)
    return positions, times_played, number_of_win

In [68]:
position_pieces = {'P':0, 'N':1, 'B':2, 'R':3, 'Q':4, 'K':5, 'p':6, 'n':7, 'b':8, 'r':9, 'q':10, 'k':11}
list_pieces = ['P', 'N', 'B', 'R', 'Q', 'K', 'p', 'n', 'b', 'r', 'q', 'k']
castle_dico = {'K':1, 'Q':0, 'k':2, 'q':3}
def from_fen_to_bitboard(fen):
    position, turn, castle, ep = fen.split(' ')
    board = [0]*12*64
    index = 0
    for p in position:
        if p in position_pieces:
            board[index+position_pieces[p]*64] = 1
            index+=1
        elif p != "/":
            index+=int(p)
    turn = 0 if turn=='w' else 0
    castle_bit=[0, 0, 0, 0]
    if(castle != '-'):
        for c in castle:
            castle_bit[castle_dico[c]] = 1
    return board+[turn]+castle_bit

def from_bitboard_to_fen(board):
    split_board = np.split(np.array(board), 12)
    fen = []
    index = 0
    for i in range(8):
        index = 0
        for k in range(8):
            piece = False
            for j in range(12):
                if split_board[j][i*8+k]:
                    piece = True
                    if index != 0:
                        fen.append(str(index))
                        index = 0
                    fen.append(list_pieces[j])
                    break
            if not piece:
                index+=1
        if index != 0:
            fen.append(str(index))
        fen.append('/')
    return fen

In [69]:
def extract_positions_color(moves, color):
    positions = []
    times_played = []
    number_of_win = []
    for win, move in tqdm(zip(games["result"], moves), total=len(moves)):
        positions, times_played, number_of_win = extract_game_fen(move, win, positions, times_played, number_of_win, color)
    return positions, times_played, number_of_win

In [72]:
def extract_games_player(games, player):
    player_file = player.replace(', ', '_').strip(',').lower()
    print(player_file)
    games = games[games["white_player"] == NAME]
    wmoves = extract_moves(games)
    wpositions, wtimes_played, wnumber_of_win = extract_positions_color(wmoves, 1)
    games = games[games["black_player"] == NAME]
    bmoves = extract_moves(games)
    bpositions, btimes_played, bnumber_of_win = extract_positions_color(wmoves, 0)
    positions = wpositions + bpositions
    times_played = wtimes_played + btimes_played
    number_of_win = wnumber_of_win + bnumber_of_win
    fen_games = pd.DataFrame.from_dict({"positions": positions, "times played":times_played, "number of win":number_of_win})
    fen_games["bitboard"] = fen_games["positions"].apply(from_fen_to_bitboard)
    fen_games["winrate"] = fen_games["number of win"] / fen_games["times played"]
    fen_games.to_csv("data/"+player_file+".csv", index=None)

In [73]:
extract_games_player(games, NAME)

carlsen_magnus


0it [00:00, ?it/s]

  0%|          | 0/1474 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1474 [00:00<?, ?it/s]